# Initialisation

In [546]:
import matplotlib
matplotlib.use('Qt5Agg')  # Utiliser le backend Qt5Agg pour windows
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import cv2
import numpy as np
import os
from PIL import Image
import lmfit
#%matplotlib tk #pour Linux

In [547]:
import scipy.ndimage as ndimage
from sklearn.cluster import DBSCAN

# Importer fichier

In [548]:
path_to_tiff = os.path.join("..", "acquisition", "video_output_carac_150ms_1im_10um.tiff")

tiff = Image.open(path_to_tiff)

# Nombre de frames pas vide

In [549]:
with Image.open(path_to_tiff) as img:
    frame_number = 0
    actual_frames = 0
    try:
        while True:
            frame_number += 1
            if np.sum(np.array(img)) != 0:
                actual_frames += 1
                
            img.seek(frame_number)
    except EOFError:
        print("All frames processed.")

actual_frames

All frames processed.


62

# 1re frame

**Le nombre de frames ignorés n'est pas pris en compte**

In [550]:
frame_index = 0
tiff.seek(frame_index)
original_image = np.array(tiff)

while np.sum(original_image) == 0:
    frame_index += 1
    tiff.seek(frame_index)
    original_image = np.array(tiff)
    print(frame_index)

# Traitement d'image

In [551]:
clahe = cv2.createCLAHE(clipLimit=10.0, tileGridSize=(30, 30))
preprocessed = clahe.apply(original_image)

blurred = cv2.medianBlur(preprocessed, 115)
preprocessed2 = cv2.subtract(preprocessed, blurred)

# Apply Non-Local Means Denoising
img = cv2.fastNlMeansDenoising(preprocessed2, None, 15, 7, 41)

# Sélection du point à tracker

In [552]:
# Display the image and let the user select a point interactively
fig, ax = plt.subplots()
ax.imshow(img, origin='lower', cmap='gray')  # Use 'gray' for better visibility of grayscale images
plt.title(f"Frame {frame_index}: Select a point")

# Ask for a point to be selected
print("Please click on the point you want to select.")
x, y = plt.ginput(1)[0]  # This will get the coordinates of the clicked point
print(f"Selected point: ({x}, {y})")
plt.close()

crop_sze_x = 150
crop_sze_y = 200

Please click on the point you want to select.
Selected point: (351.31818181818187, 324.43506493506493)


In [553]:
def crop(img, x, y, crop_size_x, crop_size_y):
    x_start = int(x - crop_size_x // 2)
    x_end = int(x + crop_size_x // 2)
    y_start = int(y - crop_size_y // 2)
    y_end = int(y + crop_size_y // 2)

    return img[y_start:y_end, x_start:x_end]

# Fit gaussien sur le point sélectionné

In [554]:
def prepare_data(x, y, z):
    return (x.flatten(), y.flatten()), z.flatten()

In [555]:
def gaussian_2d(xy, amplitude, x0, y0, sigma_x, sigma_y, offset):
    x, y = xy
    a = 1 / (2 * sigma_x**2)
    b = 1 / (2 * sigma_y**2)
    return offset + amplitude * np.exp(- (a * (x - x0)**2 + b * (y - y0)**2))

In [556]:
def localisateur_gaussien(intensity_grid, maxi):
    x = np.arange(intensity_grid.shape[0])
    y = np.arange(intensity_grid.shape[1])
    X, Y = np.meshgrid(x, y)

    # Préparer les données pour le fit
    (xdata, ydata), zdata = prepare_data(X, Y, intensity_grid)
    model = lmfit.Model(gaussian_2d)
    max_idx = np.unravel_index(np.argmax(intensity_grid), intensity_grid.shape)
    initial_x0 = x[max_idx[0]]
    initial_y0 = y[max_idx[1]]

    # Définir les paramètres du modèle
    params = model.make_params(
        amplitude=np.max(intensity_grid),
        x0=initial_x0,
        y0=initial_y0,
        sigma_x=1,
        sigma_y=1,
        offset=2
    )

    # Effectuer l'ajustement
    result = model.fit(zdata, params, xy=(xdata, ydata))

    x_position = result.params['x0'].value + maxi[0] - crop_sze_x/2
    y_position = result.params['y0'].value + maxi[1] - crop_sze_y/2

    return [x_position, y_position], result.params['sigma_x'].value, result.params['sigma_y'].value

# Process d'image (enlever le bruit)

**semble faire du trouble**

In [557]:
def denoise(image):
    clahe = cv2.createCLAHE(clipLimit=10.0, tileGridSize=(30, 30))
    preprocessed = clahe.apply(image)
    
    blurred = cv2.medianBlur(preprocessed, 115)
    preprocessed2 = cv2.subtract(preprocessed, blurred)
    
    return cv2.fastNlMeansDenoising(preprocessed2, None, 15, 7, 41)

# Passe au prochain frame

In [558]:
def next_frame(frame_i):
    frame_i += 1
    tiff.seek(frame_i)
    original_image = np.array(tiff)
    
    while np.sum(original_image) == 0:
        frame_i += 1
        tiff.seek(frame_i)
        original_image = np.array(tiff)

    return [frame_i, original_image]

Débogueur

In [559]:
def visionneur(frame):
    plt.figure(figsize=(10, 5))
    plt.clf() 
    plt.imshow(frame, origin='lower', cmap='gray')
    plt.title('Grille Zoomée avec Position')
    plt.colorbar()  
    plt.show()

# Identifier la particule et donner sa nouvelle position

In [560]:
def detect_nbre_particles(image, threshold=100, eps=15, min_samples=1):
    # 1. Appliquer un filtre de voisinage pour détecter les maxima locaux
    neighborhood_size = 3  # Taille du voisinage pour détecter les maxima locaux
    local_max = ndimage.maximum_filter(image, size=neighborhood_size)

    # 2. Comparer l'image originale et les maxima locaux pour identifier les vrais maxima
    maxima = (image == local_max) & (image > threshold)

    # 3. Extraire les coordonnées des maxima (particules)
    coordinates = np.column_stack(np.where(maxima))  # Extraire les indices des pixels maximaux
    if coordinates.shape[0] == 0:
        print("Aucune particule détectée.")
        return [], image

    # 4. Appliquer DBSCAN pour regrouper les particules proches (cluster les maxima détectés)
    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    labels = dbscan.fit_predict(coordinates)

    #5. Déterminer le nombre de particules
    unique_labels = np.unique(labels)
    nb_particules = len(unique_labels)
        
    # 6. Afficher les résultats et filtrer les particules selon leur taille et luminosité
    plt.imshow(image, origin='lower', cmap='gray')
    plt.title("Détection des particules")
    
    positions=[]
    for label in unique_labels:
        if label != -1:  # -1 correspond au bruit dans DBSCAN
            cluster_points = coordinates[labels == label]
            plt.scatter(cluster_points[:, 1], cluster_points[:, 0], label=f'Particule {label}')
#        if nb_particules > 1:
        positions.append((np.mean(cluster_points[:, 1]), np.mean(cluster_points[:, 0])))
        

    plt.colorbar()  
    plt.legend()
    plt.show()

    # Retourner les coordonnées des particules filtrées
    return nb_particules, positions

In [ ]:
def identificateur_particles(image, ref_size, ref_luminosity, threshold=100, eps=15, min_samples=1):
    # 1. Observer chaque particule approximativement
    neighborhood_size = 3  
    local_max = ndimage.maximum_filter(image, size=neighborhood_size)
    maxima = (image == local_max) & (image > threshold)

    coordinates = np.column_stack(np.where(maxima))  
    if coordinates.shape[0] == 0:
        print("Aucune particule détectée.")
        return [], image

    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    labels = dbscan.fit_predict(coordinates)
    unique_labels = np.unique(labels)

    # 2. Parcourir chaque particule et identifier laquelle est celle qu'on suit
    distances = []
    position_particules = []

    for label in unique_labels:
        if label != -1:  # -1 correspond au bruit dans DBSCAN
            cluster_points = coordinates[labels == label]
            
            size = cluster_points.shape[0]
            luminosity = np.mean(image[cluster_points[:, 0], cluster_points[:, 1]])
            size_distance = abs(size - ref_size)  
            luminosity_distance = abs(luminosity - ref_luminosity)  
            combined_distance = size_distance + luminosity_distance
            distances.append(combined_distance)
            position_particules.append((np.mean(cluster_points[:, 1]), np.mean(cluster_points[:, 0])))

    ecart = np.min(distances)
    position_best_particle = position_particules[np.argmin(distances)]  

    # 3. Afficher les résultats et filtrer les particules selon leur taille et luminosité
    plt.imshow(image, origin='lower', cmap='gray')
    plt.title("Détection des particules")
    plt.scatter(position_best_particle[0], position_best_particle[1], color='r', label=f'Particule sélectionnée')
    plt.legend()
    plt.colorbar()  
    plt.show()

    return position_best_particle, ecart

In [ ]:
def particle_tracker(image, positions, frame_index, taille_initiale, luminosité_initiale):
    # 1. Ce qu'on voit près de l'ancienne position
    image = denoise(image)
    #print(positions[frame_index][0],positions[frame_index][1])

    cropped_img = crop(image, positions[frame_index][0], positions[frame_index][1], crop_sze_x, crop_sze_y)
    
    # 2. Déterminer combien il y a de particules et laquelle est la notre
    # Utiliser une reconnaissance de particules et si plus qu'une, alors clic pour choisir
    nb_part, coordonnées = detect_nbre_particles(cropped_img)
    print(f'coordonnées des {nb_part} particules:{coordonnées}')

    if nb_part == 0:
        # Display the image and let the user select a point interactively
        fig, ax = plt.subplots()
        ax.imshow(img, origin='lower', cmap='gray') 
        plt.title(f"Frame {frame_index}: Select a point")

        # Ajouter une croix rouge à la position donnée
        cx, cy = positions[frame_index-1]
        ax.plot(cx, cy, 'rx', markersize=10)  # 'rx' pour une croix rouge, avec une taille de marqueur de 10
        
        # Afficher l'image et demander à l'utilisateur de cliquer
        print("Please click on the point you want to select.")
        x, y = plt.ginput(1)[0]  # Attente du clic de l'utilisateur (1 point)
        print(f"Selected point: ({x}, {y})")
        
        # Fermer l'affichage après sélection
        plt.close()
    elif nb_part > 1:
        imperfections, emplacement = [] , []
        for num_particule in range(nb_part):
            x_identifier=positions[frame_index][0] - crop_sze_x // 2 + coordonnées[num_particule][0]
            y_identifier=positions[frame_index][0] - crop_sze_y // 2 + coordonnées[num_particule][1]
            crop_pour_identifier = crop(image, x_identifier, y_identifier, crop_sze_x, crop_sze_y)
            imperfections.append(identificateur_particles(crop_pour_identifier, taille_initiale, luminosité_initiale)[1])
            emplacement.append(identificateur_particles(crop_pour_identifier, taille_initiale, luminosité_initiale)[0])

        qualité = np.min(imperfections)
        max_coords = emplacement[np.argmin(imperfections)]  
        print(qualité)
        nouveau_x = x_identifier - crop_sze_x // 2 + max_coords[0]
        nouveau_y = y_identifier - crop_sze_y // 2 + max_coords[1]
    else:
        cropped_img = np.array(cropped_img)
        max_index = np.argmax(cropped_img)
        max_coords = np.unravel_index(max_index, cropped_img.shape)
    #print(F'coordonnées du max dans le premier crop: ({max_coords[1]},{max_coords[0]})')
    #if frame_index>1:
    #    visionneur(cropped_img)

        nouveau_x = positions[frame_index][0] - crop_sze_x // 2 + max_coords[1] #Oui c'est inversé à cause du unravel juste avant
        nouveau_y = positions[frame_index][1] - crop_sze_y // 2 + max_coords[0]

    #print(f'(x,y) central du second crop: {nouveau_x, nouveau_y}')
    
    # 3. Crop atour de notre particule et fit dessus
    second_crop = crop(image, nouveau_x, nouveau_y, crop_sze_x, crop_sze_y)    # Re-crop autour d'une seule particule

    #if frame_index>1:
    #    visionneur(second_crop)


    result_fit = localisateur_gaussien(second_crop, [positions[frame_index][0], positions[frame_index][1]])
    #print(f'écart-type: ({result_fit[1]}, {result_fit[2]})')

    x_new, y_new = result_fit[0][0], result_fit[0][1]
    #print(f'Positions finales de l index {frame_index}: ({x_new}, {y_new}) ')
    return [result_fit, cropped_img, (x_new, y_new), (result_fit[1], result_fit[2])]

# Main loop

In [563]:
position_list = [(x,y),(x,y)]
sigma_list = []
crop_frames = []
big_frames = []
taille_initiale, luminosité_initiale = 1, 2 # À faire une fonction pour les obtenir pour le 1er frame où on clique
for _ in range(15 - 1):
    print(f'frame index: {frame_index} et len(positions): {len(position_list)}')
    data = particle_tracker(img, position_list, frame_index, taille_initiale, luminosité_initiale)
    position_list.append(data[2])
    sigma_list.append(data[3])
    frame_index, img = next_frame(frame_index)
    big_frames.append(img)
    crop_frames.append(data[1])

frame index: 0 et len(positions): 2
coordonnées des 1 particules:[(72.85185185185185, 99.51851851851852)]
frame index: 1 et len(positions): 3
coordonnées des 1 particules:[(73.74137931034483, 76.05172413793103)]
frame index: 2 et len(positions): 4
coordonnées des 1 particules:[(61.45652173913044, 76.04347826086956)]
frame index: 3 et len(positions): 5
coordonnées des 1 particules:[(60.81944444444444, 56.138888888888886)]
frame index: 4 et len(positions): 6
coordonnées des 1 particules:[(36.096774193548384, 53.354838709677416)]
frame index: 5 et len(positions): 7
coordonnées des 1 particules:[(30.59259259259259, 40.18518518518518)]
frame index: 6 et len(positions): 8
coordonnées des 1 particules:[(15.407407407407407, 30.333333333333332)]
frame index: 7 et len(positions): 9
coordonnées des 2 particules:[(18.89189189189189, 15.91891891891892), (14.666666666666666, 198.55555555555554)]
Aucune particule détectée.
Aucune particule détectée.
frame index: 8 et len(positions): 10
coordonnées de

TypeError: '>' not supported between instances of 'list' and 'int'

# Résultats

In [ ]:
position_list

[(339.62987012987014, 321.51298701298697),
 (339.62987012987014, 321.51298701298697),
 (353.9609417175827, 298.8538643240032),
 (357.2262420144212, 294.9758671194706),
 (381.1268493256229, 277.65813914941583),
 (389.3034867524197, 269.2643474736124),
 (405.8850136101475, 254.21155764467795),
 (405.0476224867656, 247.65745380198325),
 (430.7334583891503, 227.30095986741156),
 (406.61363385679226, 222.65757435437558),
 (434.2469460495615, 202.18945503032575),
 (411.0985049644702, 197.6267127453325),
 (592.1215414456447, 177.25492279867706),
 (414.8929804420299, 172.51260320608685),
 (679.1301592376135, 226.92262239757713),
 (416.671859413107, 147.51318644155356)]

In [ ]:
sigma_list

[(-445.08438798145056, -12.489978937001327),
 (-296.6137833981995, -12.459435976439936),
 (-295.26881469721764, -12.107025736064102),
 (337.79211261168274, 10.157853407802826),
 (0.246409040029674, 0.03313141480023485),
 (-307.4330065140979, -10.053056826186406),
 (277.2033215884723, 10.352039948273548),
 (10.97726150973065, -0.0066868388309803024),
 (14.47924423442618, -0.15603103349767197),
 (11.768027634469345, 0.15928759451045624),
 (-420.8337727242347, -12.771863549284278),
 (14.85296491514032, -0.16470635267126887),
 (9.278410903553919, 0.3237131841917487),
 (11.257959257803147, -0.050979141064037906)]

In [ ]:
# Supposons que position_list est une liste de tuples (x, y)
x_plt, y_plt = zip(*position_list)

# Créer le graphique
plt.figure(figsize=(10, 8))  # Optionnel : ajuster la taille de la figure
plt.plot(x_plt, y_plt, marker='o', linestyle='-', color='b', label='Connected Points')

# Inverser l'axe Y
plt.gca().invert_yaxis()

# Définir les limites de la grille pour correspondre à une grille de 1440x1080
plt.xlim(0, 1440)
plt.ylim(0, 1080)

# Ajouter les labels et le titre
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.title('2D Connected Points Plot')
plt.legend()

# Optionnel : ajouter des lignes de grille
plt.grid(True)

# Afficher le graphique
plt.show()

## anim toutes les particules

In [ ]:
fig, ax = plt.subplots()
img = ax.imshow(big_frames[0], origin='lower', cmap='gray', animated=True)

# Créer un scatter plot animé avec des coordonnées (x_plt, y_plt)
scat = ax.scatter(x_plt, y_plt, color='b', marker='o', label='Points', animated=True)

# Fonction de mise à jour
def update(frame):
    img.set_array(frame)  # Mettre à jour l'image à chaque frame
    return img, scat  # Retourner à la fois l'image et le scatter

# Créer l'animation
ani = animation.FuncAnimation(fig, update, frames=big_frames, interval=50, blit=True)

# Afficher l'animation
plt.show()


## anim crop

In [ ]:
fig, ax = plt.subplots()
img = ax.imshow(crop_frames[0], origin='lower', cmap='gray', animated=True)


def update(frame):
    img.set_array(frame)
    return img,
    
ani = animation.FuncAnimation(fig, update, frames=crop_frames, interval=50, blit=True)
plt.show()